In [ ]:
from selenium.common.exceptions import NoSuchElementException
import sys
import codecs
import time
from datetime import date, timedelta

today = date.today()
yesterday = today - timedelta(days=1)
d = today.strftime("%b %d")
d1 = yesterday.strftime("%b %d")

separator = "	"
result = "\"Text\""+separator+"\"Title\""+separator+"\"Rating\""+separator+"\"DateOfExperience\""+separator+"\"DateOfReview\""+separator+"\"Username\""+separator+"\"UserLocation\""+separator+"\"UserContribution\""+separator+"\"UserLikes\""
def check_exists_element(sel, path):
    try:
        sel.find_element_by_css_selector(path)
    except NoSuchElementException:
        return False
    return True

def get_data(i, pagetype):
    data_lists = driver.find_elements_by_css_selector("div.location-review-card-Card__ui_card--2Mri0.location-review-card-Card__card--o3LVm.location-review-card-Card__section--NiAcw")[i:]
    print("step0:"+str(int(time.time()*1000)))
    item_no=0
    for data in data_lists:
        username = data.find_element_by_css_selector("a.ui_header_link.social-member-event-MemberEventOnObjectBlock__member--35-jC").text

        title = data.find_element_by_css_selector("a.location-review-review-list-parts-ReviewTitle__reviewTitleText--2tFRT").text
        print("step1:"+str(int(time.time()*1000)))
        readmore = data.find_element_by_css_selector("div._36B4Vw6t")
        if pagetype==1:
            if item_no==0:
                readmore.click()
                time.sleep(2)
            print("step2:"+str(int(time.time()*1000)))
        elif pagetype==2:
            readmore.click()
            print("step2:"+str(int(time.time()*1000)))
            time.sleep(1)
        
        item_no+=1
        print("step3:"+str(int(time.time()*1000)))
        review_content = data.find_element_by_css_selector("q.location-review-review-list-parts-ExpandableReview__reviewText--gOmRC").text
        review_content = " ".join(review_content.split("\n"))

        review_date = data.find_element_by_css_selector("div.social-member-event-MemberEventOnObjectBlock__event_type--3njyv").text
        review_date = review_date[(review_date.find("wrote a review")+15):]
        if review_date=="Today": review_date=d
        elif review_date=="Yesterday": review_date=d1

        experience_date = ""
        if check_exists_element(data,"span.location-review-review-list-parts-EventDate__event_date--1epHa"):
            experience_date = data.find_element_by_css_selector("span.location-review-review-list-parts-EventDate__event_date--1epHa").text
            experience_date = experience_date[20:]
        
        bubble = data.find_element_by_css_selector("span.ui_bubble_rating").get_attribute("class").split()
        bubble = bubble[1].split("_")
        rating = bubble[1]
        
        user_location=""
        if check_exists_element(data,"span.social-member-MemberHeaderStats__hometown_stat_item--231iN"):
            user_location = data.find_element_by_css_selector("span.social-member-MemberHeaderStats__hometown_stat_item--231iN").text
        print("step4:"+str(int(time.time()*1000)))
        votes = ""
        contributions = ""
        if check_exists_element(data, "span.social-member-MemberHeaderStats__stat_item--34E1r"):
            user_infos = data.find_elements_by_css_selector("span.social-member-MemberHeaderStats__stat_item--34E1r")
            for user_info in user_infos:
                info = user_info.text.split()
                count = info[0]
                txt = info[1]
                if txt.find("helpful")!=-1:
                    votes = count
                elif txt.find("contribution")!=-1:
                    contributions = count
        content = "\""+review_content+"\""+separator+"\""+title+"\""+separator+"\""+rating+"\""+separator+"\""+experience_date+"\""+separator+"\""+review_date+"\""+separator+"\""+username+"\""+separator+"\""+user_location+"\""+separator+"\""+contributions+"\""+separator+"\""+votes+"\""
        
        f.write("\n"+content)
        print("step5:"+str(int(time.time()*1000)))

driver = webdriver.Chrome('/Users/maxrojas/Downloads/chromedriver')  # Optional argument, if not specified will search path.
driver.maximize_window()
driver.set_window_position(-10000,0) 
driver.get('hhttps://www.tripadvisor.com/Attraction_Review-g189852-d208370-Reviews-Stockholm_City_Hall-Stockholm.html')
#driver.get('https://www.tripadvisor.com/Attraction_Review-g189852-d4182487-Reviews-ABBA_The_Museum-Stockholm.html')

time.sleep(25)

page_type=1
if check_exists_element(driver, "a.ui_button.nav.next.primary"):
    page_type=1
elif check_exists_element(driver, "div.location-review-review-list-parts-ReviewScrollController__show_more--3zk4I"):
    page_type=2

print("page-type:"+str(page_type))
filename = "cityhall.csv"
f= codecs.open(filename,"w+")
f.write(result)

i=0
pages = 0
while(True):
    pages+=1
    get_data(i, page_type)
    if check_exists_element(driver, "a.ui_button.nav.next.primary"):
        next_button = driver.find_element_by_css_selector("a.ui_button.nav.next.primary")
        next_button.click()
        time.sleep(5)
    elif check_exists_element(driver, "div.location-review-review-list-parts-ReviewScrollController__show_more--3zk4I"):
        read_more = driver.find_element_by_css_selector("div.location-review-review-list-parts-ReviewScrollController__show_more--3zk4I")
        read_more.click()
        time.sleep(3)
        i+=5
    else:
        print("end")
        break
    print("page:"+str(pages))

f.close() 

driver.quit()